In [5]:
%pip install git+https://github.com/rlworkgroup/metaworld.git@master#egg=metaworld

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/rlworkgroup/metaworld.git (to revision master) to /tmp/pip-install-d14b6t7h/metaworld_f9c023d73be344238832f89f2f3f2d6b
  Running command git clone --filter=blob:none --quiet https://github.com/rlworkgroup/metaworld.git /tmp/pip-install-d14b6t7h/metaworld_f9c023d73be344238832f89f2f3f2d6b
  Resolved https://github.com/rlworkgroup/metaworld.git to commit 04be337a12305e393c0caf0cbf5ec7755c7c8feb
  Preparing metadata (setup.py) ... done
  Using cached gym-0.26.2-py3-none-any.whl
  Using cached mujoco_py-2.1.2.14-py3-none-any.whl (2.4 MB)
  Using cached gym_notices-0.0.8-py3-none-any.whl (3.0 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached fasteners-0.18-py3-none-any.whl (18 kB)
  Using cached glfw-2.5.9-py2.py27.py3.py30.py31.py32.py33.py34.py35.py36.py37.py38-none-manylinux2014_x86_64.whl (207 kB)
  Using cached Cython-0.29.34-cp310-cp310-manylinux_2_17_x8

In [6]:
import metaworld

Compiling /home/dema/.local/lib/python3.10/site-packages/mujoco_py/cymj.pyx because it depends on /home/dema/.local/lib/python3.10/site-packages/mujoco_py/pxd/mujoco.pxd.
[1/1] Cythonizing /home/dema/.local/lib/python3.10/site-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
INFO: C compiler: x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC

creating /home/dema/.local/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxgpuextensionbuilder
creating /home/dema/.local/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxgpuextensionbuilder/temp.linux-x86_64-3.10
creating /home/dema/.local/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.1.2.14_310_linuxgpuextensionbuilder/temp.linux-x86_64-3.10/

In [8]:
import numpy as np
import random 

In [11]:
class fake_dynamics:

    def __init__(self):
        ml1 = metaworld.ML1('pick-place-v2')
        task = random.choice(ml1.train_tasks)
        self.env = ml1.train_classes['pick-place-v2']()
        self.env.set_task(task)

    def forward(self, action):
        return_val = self.env.step(action)
        next_state, reward = return_val[:2]
        return next_state, reward



In [12]:
class CEM_opt:
    '''
    gradient free optimizer
    We want to use it for get the action distribution
    QUESTION: HOW IS RELATED TO THE ORIGINAL STATE?
    '''

    def __init__(self, cost_funct,
                 population_shape,   # action dim * task horizon
                 numb_population: int = 10,
                 numb_elite: int = 5):
        self.cost_funct = cost_funct
        self.numb_elite = numb_elite
        self.numb_population = numb_population
        self.population_shape = population_shape
        self.mean_vect = np.zeros(self.population_shape)
        self.std = np.ones(self.population_shape)

        self.population = [
            self.mean_vect + np.random.rand(self.population_shape)*self.std
            for i in range(self.numb_population)
        ]

    def update(self):
        rewards = [self.cost_funct(weights) for weights in self.population]
        elite_idxs = np.array(rewards).argsort()[-self.numb_elite:]
        elite_weights = [self.population[idx] for idx in elite_idxs]
        self.mean_vect = np.array(elite_weights).mean()
        self.std = np.array(elite_weights).std()

        self.population = [
            self.mean_vect + np.random.rand(self.population_shape)*self.std
            for i in range(self.numb_population)
        ]

    def get_solution(self):
        return self.mean_vect


In [14]:
class Planner:

    def __init__(self, dynamics, action_dim=4, plan_hor=20, popsize=500):
        #self.dynamics = dynamics
        self.dynamics = fake_dynamics()

        self.action_dim = action_dim
        self.action_buffer = np.array([]).reshape(0, self.action_dim)

        self.plan_hor = plan_hor
        self.cem_optimizer = CEM_opt(self.rollout_funct,
                                     population_shape=self.action_dim*self.plan_hor
                                     )

    def rollout_funct(self, actions):
        '''
        :param actions: 1D vector : action sequence
        :return: cumulative rewards
        '''
        reward = 0
        for a in actions:
            next_s, r = self.dynamics.forward(a)
            reward += r
        return reward

    def get_action(self):
        if self.action_buffer.shape[0] > 0:

            action = self.action_buffer[0]
            self.action_buffer = self.action_buffer[1:]
            return action

        solution = self.cem_optimizer.get_solution()
        # manca l'update del cem : non spetta al planner
        return self.get_action()
        # add solution to action_buffer

        # to understand : role of previous solution of original code
        # in the main code the part of self.prev_sol = np.concat Put the last action = 0
